### Description

- Парсер пульса с прокруткой (для сбора начальной базы данных с помощью selenium)
- Преобразование даннных в pandas.dataframe
- В конце первые попытки подвязать gpt

### Libraries

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import emoji
import re
from datetime import datetime, timedelta
import json
import pandas as pd
from openai import OpenAI
import settings

### Parser

In [323]:
class Article:
    month_dict = {
    'января': '01',
    'февраля': '02',
    'марта': '03',
    'апреля': '04',
    'мая': '05',
    'июня': '06',
    'июля': '07',
    'августа': '08',
    'сентября': '09',
    'октября': '10',
    'ноября': '11',
    'декабря': '12'
    }
    
    def __init__(self, date, text):#, likes):
        self.date = self.fix_dates(date)
        self.text = self.clean_text(text)
        # self.likes = likes

    def clean_text(self, text):
        emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F700-\U0001F77F"  # alchemical symbols
                                u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                                u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                                u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                                u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                                u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                                u"\U00002702-\U000027B0"  # Dingbats
                                u"\U000024C2-\U0001F251" 
                                "]+", flags=re.UNICODE)
        clean_text = emoji_pattern.sub(r'', text)
        return clean_text
    
    def fix_dates(self, date_string):
        if 'Сегодня' in date_string:
            result = datetime.combine(datetime.now().date(), datetime.strptime(date_string.split(" ")[-1], '%H:%M').time()).strftime("%Y-%m-%d %H:%M")
        elif 'Вчера' in date_string:
            result = datetime.combine(datetime.now().date() - timedelta(days=1), datetime.strptime(date_string.split(" ")[-1], '%H:%M').time()).strftime("%Y-%m-%d %H:%M")
        else:
            result_list = date_string.split(" ")
            result_list[1] = self.month_dict[result_list[1]]
            result_list.pop(3)
            date_str = ' '.join(result_list)
            # result = date_str
            date_format = '%d %m %Y %H:%M'
            result = datetime.strptime(date_str, date_format).strftime("%Y-%m-%d %H:%M:%S")
        return result

class Author:

    def get_subscribers():
        pass
    def get_profitability():
        pass
    def get_base_url(self, name, start= settings.BASE_URL):
        return start+name
    def __init__(self, name: str):
        self.name = name
        self.base_url = self.get_base_url(name)

        self.subscribers, self.size, self.month_activity, self.profitability, self.articles = self.get_full_data(self.base_url)


    def get_full_data(self, url):
        driver = webdriver.Chrome()
        driver.get(url)
        try:
            # for i in range(2):  
            #     print('start', i)
            #     time.sleep(7)
            #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1000);")
            #     time.sleep(1)

            page_source = driver.page_source
        
            soup = BeautifulSoup(page_source, 'html.parser')
            data = soup.find_all('div', {'data-qa-tag':'PulsePost'})
            profile_info = soup.find_all('div', {'class':'pulse-profilepage__dej4vS'})


            size = list(profile_info[0])[1].get_text().replace('\xa0', ' ').replace('₽', '').split(" ", 1)[0] + " " + list(profile_info[0])[1].get_text().replace('\xa0', ' ').replace('₽', '').split(" ", 1)[1].replace(" ", "")
            month_activity = int(list(profile_info[1])[1].get_text().split(" ")[0])
            profitability = float(list(profile_info[2])[1].get_text().replace(',', '.').replace(' ', '')[:-1])

            subscribers = int(soup.find_all('span', {'class':'pulse-profilepage__hQEzFm'})[0].get_text().replace(' ', ''))

            articles = []
            print(f"Загружено {len(data)} постов")

            for post in data:
                text = post.find_all('div', {'class':'pulse-profilepage__fGGBmY'})[0].get_text()
                date = post.find_all('div', {'pulse-profilepage__cSULlZ'})[0].get_text()
                # likes = post.find_all('div', {'pulse-profilepage__ei8mu9'})[0].get_text()

                article = Article(date, text)#, likes)
                articles.append(article)

            return subscribers, size, month_activity, profitability, articles
        finally:
            driver.quit()
        
class Authors:
    def __init__(self, authors_list=[]):
        self.authors_list = authors_list

    def add_author(self, author):
        self.authors_list.append(author)

    def save_to_json(self, filename):
        author_data_list = []
        for author in self.authors_list:
            author_data = {
                "name": author.name,
                "base_url": author.base_url,
                "subscribers": author.subscribers,
                "size": author.size,
                "month_activity": author.month_activity,
                "profitability": author.profitability,
                "articles": [{"date": article.date, "text": article.text} for article in author.articles],
            }
            author_data_list.append(author_data)

        with open(filename, "w", encoding="utf-8") as json_file:
            json.dump(author_data_list, json_file, ensure_ascii=False, indent=4)

In [348]:
names = ["artydevCo",
"ALL_TIME_TRADING",
"Gaong",
"Vladislavzz",
"CyberMoneyFunnyPunk"]

### Run and save

In [349]:
l = []
for name in names:
    l.append(Author(name))
a = Authors(l)
a.save_to_json("authors_data.json")

Загружено 30 постов
Загружено 30 постов
Загружено 30 постов
Загружено 30 постов
Загружено 30 постов


### Create DataFrame

In [12]:
df = pd.read_json("authors_data.json")
df = df.explode('articles')
df.reset_index(inplace=True, drop=True)

df['date'] = df['articles'].apply(lambda x: x.pop('date'))
df['text'] = df['articles'].apply(lambda x: x.pop('text'))
df.drop(columns=['articles'], inplace=True)



### GPT

In [24]:
file_path = 'question.txt'

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()
except FileNotFoundError:
    print(f"The file {file_path} does not exist.")
except Exception as e:
    print(f"An error occurred: {e}")


In [25]:
print(file_content+df.loc[0, 'text'])

Напиши три ответа на основании предложенного текста, ответы должны быть односложными:
1. напиши только рассматриваемый в тексте ticket на бирже(обычно название идет после знака "$"). Если текст говорит о рынке в целом, выведи "all", а если на основании текста тикет не понятен, напиши 'none' в ответах на все три вопроса
2. на основании текста от -1 до 1 оцени вероятность и направление движения цены, где -1 - цена точно пойдет вниз, а +1 - цена точно пойдет вверх. Напиши только число
3. на основании текста ответь, какая длительность у прогноза. Если до 2 недель, выведи 0; если от 2 недель до 3 месяцев то выведи 1; если длительность прогноза до полугода, выведи 2; если больше полугода, выведи 3, если непонятно, выведи -1. Нужно только одно число, без пояснений
предложенный текст: Товарищи инвесторы, а что происходит с $OZON ?
Есть какие-то инсайды на тему будущего отчёта, откуда такой рост за последние недели?
Держу +120% в стратегии 


&Capital Managament


In [20]:
file_content

'Напиши три ответа на основании предложенного текста, ответы должны быть односложными:\n1. напиши только рассматриваемый в тексте ticket на бирже(обычно название идет после знака "$"). Если текст говорит о рынке в целом, выведи "all", а если на основании текста тикет не понятен, напиши \'none\' в ответах на все три вопроса\n2. на основании текста от -1 до 1 оцени вероятность и направление движения цены, где -1 - цена точно пойдет вниз, а +1 - цена точно пойдет вверх. Напиши только число\n3. на основании текста ответь, какая длительность у прогноза. Если до 2 недель, выведи 0; если от 2 недель до 3 месяцев то выведи 1; если длительность прогноза до полугода, выведи 2; если больше полугода, выведи 3. Нужно только одно число, без пояснений\nпредложенный текст: '

In [26]:
query_text = file_content+df.loc[0, 'text']

In [22]:
DS_API_KEY = "sk-a8b2b1aea0f940fbb135a36e7ae16302"

client = OpenAI(api_key=DS_API_KEY, base_url="https://api.deepseek.com/v1")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "user", "content": query_text},
    ]
)

print(response.choices[0].message.content)


 1. OZON
2. 0.6 (утверждения о надежных "инсайдах" и стратегии +120% указывают на возможное повышение цены)
3. 1 (нет прямых указаний на определённую продолжительность прогноза, однако ожидание будущего отчёта и стратегия трейдинга указывают на текущий период от 2 недель до 3 месяцев)


In [27]:
OXYGEN_API_KEY = "oxy-UCnZ5b2l0nDkyttL2iBNJA2ylGjK3MXzuEHOY0nJpFN5R"

client = OpenAI(api_key=OXYGEN_API_KEY,base_url="https://app.oxyapi.uk/v1")

stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": query_text}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

1. $OZON
2. +1
3. -1